# REMEMBER THESE NOTEBOOKS AREN'T SAVED TO GITHUB. MANUALLY DOWNLOAD THEM

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


In [2]:
!git clone https://github.com/Locrian24/seng474-term-project.git

fatal: destination path 'seng474-term-project' already exists and is not an empty directory.


In [3]:
!cd seng474-term-project/ && git pull

Already up to date.


In [4]:
import sys
sys.path.insert(0, '/content/seng474-term-project')

In [5]:
from encode_to_seq import Encode2Seq

In [6]:
# Global variables rn for testing

ANNOTATIONS = '/content/seng474-term-project/data/gvcf2seq.annotation_embeddings.csv'
EMBEDDINGS = '/content/seng474-term-project/data/embeddings.txt'
REF = '/content/seng474-term-project/data/ref.seq'

In [7]:
VCF = '/content/seng474-term-project/step3/star_samples.vcf'
LABELS = '/content/seng474-term-project/step3/labels.csv'

In [8]:
np.random.seed(1337)

In [9]:
encoding = Encode2Seq(vcf=VCF, labels=LABELS, label_cols=[0, 1, 2], embedding_file=EMBEDDINGS, annotation_file=ANNOTATIONS, ref_seq=REF)

In [10]:
mask = np.all(np.isnan(encoding.y) == False, axis=1)

In [11]:
sample_y = encoding.y[mask]
sample_X = encoding.X[mask.reshape(-1, 1).any(axis=1)]
sample_names = encoding.sample_names[mask]

In [12]:
# Get valid stars
all_stars = np.array([s.split('_')[1] for s in sample_names])
stars, idx = np.unique(all_stars, return_index=True)

# Choose which stars are training and which are test: (31, 24) split
# Should be stratified with labels
train_idx, test_idx = train_test_split(idx, stratify=sample_y[idx], test_size=0.43)

# Retrieve indices of training and test stars
sample_mask = np.isin(all_stars, all_stars[train_idx])

# Split the data into the two sets
_train_X, test_X = sample_X[sample_mask], sample_X[~sample_mask]
_train_y, test_y = sample_y[sample_mask], sample_y[~sample_mask]

uncurated_X = encoding.X[(~mask).reshape(-1, 1).any(axis=1)]
uncurated_samples = encoding.sample_names[~mask]

# Split training into train + validation (10% split -> validation)
train_X, val_X, train_y, val_y = train_test_split(_train_X, _train_y, stratify=_train_y, test_size=0.1, random_state=0)

## Custom output layers to convert probabilities to sample label

https://arxiv.org/pdf/1901.07884.pdf

In [13]:
class FunctionProbToLabelLayer(tf.keras.layers.Layer):
  def __init__(self, label, **kwargs):
    self.label = label
    super(FunctionProbToLabelLayer, self).__init__(**kwargs)

## Load and fit model

In [14]:
json_file = open('/content/seng474-term-project/step_1/model.json', 'r')
loaded_model = json_file.read()
model = tf.keras.models.model_from_json(loaded_model)
model.load_weights('/content/seng474-term-project/step_1/weights.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 2970, 70)          17360     
_________________________________________________________________
batch_1 (BatchNormalization) (None, 2970, 70)          280       
_________________________________________________________________
relu_1 (ReLU)                (None, 2970, 70)          0         
_________________________________________________________________
maxpooling_1 (MaxPooling1D)  (None, 990, 70)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 196, 46)           35466     
_________________________________________________________________
batch_2 (BatchNormalization) (None, 196, 46)           184       
_________________________________________________________________
relu_2 (ReLU)                (None, 196, 46)           0

In [15]:
model.pop()
model.pop()
model.pop()
model.trainable = False

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 2970, 70)          17360     
_________________________________________________________________
batch_1 (BatchNormalization) (None, 2970, 70)          280       
_________________________________________________________________
relu_1 (ReLU)                (None, 2970, 70)          0         
_________________________________________________________________
maxpooling_1 (MaxPooling1D)  (None, 990, 70)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 196, 46)           35466     
_________________________________________________________________
batch_2 (BatchNormalization) (None, 196, 46)           184       
_________________________________________________________________
relu_2 (ReLU)                (None, 196, 46)           0

In [16]:
inputs = tf.keras.Input(shape=(14868, 13))
x = model(inputs, training=False)
x = tf.keras.layers.Dense(32, activation=tf.keras.activations.relu, kernel_initializer=tf.keras.initializers.VarianceScaling(mode='fan_avg', distribution='uniform'), name = "dense_5")(x)
x = tf.keras.layers.Dropout(rate=0.03, name="dropout_4")(x)
x = tf.keras.layers.Dense(1, activation=tf.keras.activations.linear, kernel_initializer=tf.keras.initializers.VarianceScaling(mode='fan_avg', distribution='uniform'), name = "dense_6")(x)
outputs = tf.keras.layers.Dense(2, activation=tf.keras.activations.sigmoid, kernel_initializer=tf.keras.initializers.VarianceScaling(mode='fan_avg', distribution='uniform'), name = "final_dense")(x)

final_model = tf.keras.Model(inputs, outputs)
final_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 14868, 13)]       0         
_________________________________________________________________
sequential (Sequential)      (None, 92)                68332     
_________________________________________________________________
dense_5 (Dense)              (None, 32)                2976      
_________________________________________________________________
dropout_4 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 33        
_________________________________________________________________
final_dense (Dense)          (None, 2)                 4         
Total params: 71,345
Trainable params: 3,013
Non-trainable params: 68,332
_____________________________________________________

In [17]:
_train_ds = tf.data.Dataset.from_tensor_slices((train_X, train_y))
train_ds = _train_ds.cache().shuffle(32).batch(32).prefetch(buffer_size=10)

_val_ds = tf.data.Dataset.from_tensor_slices((val_X, val_y))
val_ds = _val_ds.cache().shuffle(32).batch(32).prefetch(buffer_size=10)

_test_ds = tf.data.Dataset.from_tensor_slices((test_X, test_y))
_test_ds = _test_ds.cache().prefetch(buffer_size=10)

In [18]:
final_model.compile(
    optimizer=tf.keras.optimizers.Adam(0.01),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=[tf.keras.metrics.BinaryAccuracy()]
)

epochs = 20
final_model.fit(train_ds, epochs=epochs, validation_data=val_ds)

Epoch 1/20
5/5 [==============================] - 2s 159ms/step - loss: 0.6612 - binary_accuracy: 0.5528 - val_loss: 0.4865 - val_binary_accuracy: 0.5625
Epoch 2/20
5/5 [==============================] - 1s 104ms/step - loss: 0.5078 - binary_accuracy: 0.6373 - val_loss: 0.5151 - val_binary_accuracy: 0.7188
Epoch 3/20
5/5 [==============================] - 1s 103ms/step - loss: 0.5037 - binary_accuracy: 0.8134 - val_loss: 0.4445 - val_binary_accuracy: 0.8125
Epoch 4/20
5/5 [==============================] - 1s 106ms/step - loss: 0.4184 - binary_accuracy: 0.8345 - val_loss: 0.4220 - val_binary_accuracy: 0.8750
Epoch 5/20
5/5 [==============================] - 1s 104ms/step - loss: 0.4015 - binary_accuracy: 0.8380 - val_loss: 0.4252 - val_binary_accuracy: 0.8125
Epoch 6/20
5/5 [==============================] - 1s 104ms/step - loss: 0.3561 - binary_accuracy: 0.8768 - val_loss: 0.4125 - val_binary_accuracy: 0.8125
Epoch 7/20
5/5 [==============================] - 1s 105ms/step - loss: 0.34

In [19]:
model.trainable = True
final_model.compile(
    tf.keras.optimizers.Adam(1e-5),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=[tf.keras.metrics.BinaryAccuracy()]
)

epochs = 7
final_model.fit(train_ds, epochs=epochs, validation_data=val_ds)

Epoch 1/7
5/5 [==============================] - 3s 358ms/step - loss: 0.2206 - binary_accuracy: 0.9120 - val_loss: 0.2869 - val_binary_accuracy: 0.8438
Epoch 2/7
5/5 [==============================] - 2s 317ms/step - loss: 0.2242 - binary_accuracy: 0.8979 - val_loss: 0.2859 - val_binary_accuracy: 0.8750
Epoch 3/7
5/5 [==============================] - 2s 320ms/step - loss: 0.2092 - binary_accuracy: 0.9155 - val_loss: 0.2837 - val_binary_accuracy: 0.8438
Epoch 4/7
5/5 [==============================] - 2s 320ms/step - loss: 0.2149 - binary_accuracy: 0.9014 - val_loss: 0.2835 - val_binary_accuracy: 0.8438
Epoch 5/7
5/5 [==============================] - 2s 320ms/step - loss: 0.2149 - binary_accuracy: 0.9155 - val_loss: 0.2798 - val_binary_accuracy: 0.8438
Epoch 6/7
5/5 [==============================] - 2s 318ms/step - loss: 0.2164 - binary_accuracy: 0.9085 - val_loss: 0.2802 - val_binary_accuracy: 0.8438
Epoch 7/7
5/5 [==============================] - 2s 321ms/step - loss: 0.2138 - bi

In [20]:
final_model.evaluate(test_X, test_y)

2/2 [==============================] - 0s 99ms/step - loss: 0.3210 - binary_accuracy: 0.8103


[0.32100391387939453, 0.8103448152542114]

In [21]:
preds = final_model.predict(test_X)

In [22]:
def get_functions(pred):
  cutpoint_1 = 0.4260022
  cutpoint_2 = 0.7360413

  cut1 = np.greater(pred[:, 0], [cutpoint_1])
  cut2 = np.greater(pred[:, 1], [cutpoint_2])

  functions = []
  for i in range(pred.shape[0]):
    if cut1[i] == True and cut2[i] == True:
      functions.append("Normal")
    elif cut1[i] == True and cut2[i] == False:
      functions.append("Decreased Function")
    else:
      functions.append("No function")

  return np.array(functions)

In [23]:
predictions = get_functions(preds)
true_labels = get_functions(test_y)

In [24]:
np.sum(predictions == true_labels) / len(true_labels)

0.8620689655172413

In [25]:
_uncurated_predictions = final_model.predict(uncurated_X)
uncurated_predictions = get_functions(_uncurated_predictions)

In [26]:
list(zip(uncurated_samples, uncurated_predictions))

[('CYP2D6_102_001', 'Decreased Function'),
 ('CYP2D6_102_vcf', 'Normal'),
 ('CYP2D6_103_001', 'Decreased Function'),
 ('CYP2D6_103_vcf', 'Normal'),
 ('CYP2D6_104_001', 'Decreased Function'),
 ('CYP2D6_104_vcf', 'Decreased Function'),
 ('CYP2D6_105_001', 'No function'),
 ('CYP2D6_105_vcf', 'Decreased Function'),
 ('CYP2D6_106_001', 'Decreased Function'),
 ('CYP2D6_106_002', 'Normal'),
 ('CYP2D6_106_vcf', 'Decreased Function'),
 ('CYP2D6_107_001', 'Normal'),
 ('CYP2D6_107_vcf', 'Decreased Function'),
 ('CYP2D6_108_001', 'Normal'),
 ('CYP2D6_108_vcf', 'Normal'),
 ('CYP2D6_109_001', 'Normal'),
 ('CYP2D6_109_vcf', 'Decreased Function'),
 ('CYP2D6_110_001', 'Decreased Function'),
 ('CYP2D6_110_vcf', 'Decreased Function'),
 ('CYP2D6_111_001', 'Decreased Function'),
 ('CYP2D6_111_002', 'Decreased Function'),
 ('CYP2D6_111_vcf', 'Decreased Function'),
 ('CYP2D6_112_001', 'Normal'),
 ('CYP2D6_112_vcf', 'Normal'),
 ('CYP2D6_113_001', 'Decreased Function'),
 ('CYP2D6_113_vcf', 'Decreased Function'